In [6]:
import numpy as np
import nibabel as nib
import torch
import pandas as pd
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {device}")


# Read the CSV file into a DataFrame
df = pd.read_csv('data/TrainValid_ground_truth.csv')

# Filter the DataFrame to exclude rows where the 'id' contains an underscore
filtered_df = df[~df['id'].str.contains('_')]

# Get the 'id' column as a list
id_list = filtered_df['id'].tolist()

image_directory = 'data/1_Train,Valid_Image/'
mask_directory = 'data/2_Train,Valid_Mask'
matching_images = []
matching_masks = []


#print(id_list[0])

matching_files = []
for i, items in enumerate(id_list):
    id_match = id_list[i]
    for filename in os.listdir(image_directory):
        if filename.endswith('nii.gz') and id_match in filename:
            matching_images.append(os.path.join(image_directory, filename))
    for filename in os.listdir(mask_directory):
        if filename.endswith('nii.gz') and id_match in filename:
            matching_masks.append(os.path.join(mask_directory, filename))




# Load the NIfTI file
nii_mask_path = 'data/2_Train,Valid_Mask/Zx00AD16F8B97A53DE6E7CFE260BDF122F0E655659A3DF1628_label.nii.gz'
nii_mask = nib.load(nii_mask_path)
nii_img_path = 'data/1_Train,Valid_Image/Zx00AD16F8B97A53DE6E7CFE260BDF122F0E655659A3DF1628.nii.gz'
nii_img = nib.load(nii_img_path)

# Get the data as a numpy array
mask_data = nii_mask.get_fdata()
img_data = nii_img.get_fdata()

slices_with_1 = [i for i in range(mask_data.shape[-1]) if 1 in np.unique(mask_data[..., i])]
import matplotlib.pyplot as plt

def apply_window(image, level, width):
    lower = level - width / 2
    upper = level + width / 2
    return np.clip(image, lower, upper)

def normalize_01(image, WL=45, WW=300):
    """Normalize the volume"""
    CT_min = WL - WW/2
    CT_min = -200
    CT_max = WL + WW/2

    image = (image - CT_min) / (CT_max - CT_min)
    image[image > 1] = 1.0
    image[image < 0] = 0.0

    return image

def normalize(image, WL=45, WW=300):
    """Normalize the volume"""
    CT_min = WL - WW/2
    CT_min = -200
    CT_max = WL + WW/2

    print(CT_min, CT_max)
    print(image.min(), image.max())

    image[image > CT_max] = CT_max
    image[image < CT_min] = CT_min

    return image


def plot_image_and_mask(slice_index, level, width):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))


    windowed_image = apply_window(img_data[:, :, slice_index], level, width)
    n01 = normalize_01(img_data[:, :, slice_index], level, width)
    n = normalize(img_data[:, :, slice_index], level, width)

    # Plot image slice
    axes[0][0].imshow(windowed_image, cmap='gray')
    axes[0][0].set_title(f'Image Slice {slice_index}')
    print(windowed_image[:10])
    axes[0][0].axis('off')

    # Plot mask slice
    axes[0][2].imshow(mask_data[:, :, slice_index], cmap='gray')
    axes[0][2].set_title(f'Mask Slice {slice_index}')
    axes[0][2].axis('off')

    axes[0][1].imshow(img_data[:, :, slice_index], cmap='gray')
    axes[0][1].imshow(mask_data[:, :, slice_index], cmap='gray', alpha=0.5)
    axes[0][1].set_title(f'Slice {slice_index}')
    axes[0][1].axis('off')

    axes[1][0].imshow(n, cmap='gray')
    axes[1][0].set_title(f'Slice {slice_index} Normalized')
    axes[1][0].axis('off')
    print(n[:10])

    axes[1][1].imshow(n01, cmap='gray')
    axes[1][1].set_title(f'Slice {slice_index} Normalized 0-1')
    axes[1][1].axis('off')
    print(n01[:10])
    axes[1][2].axis('off')

    plt.show()

import ipywidgets as widgets

num_slices = img_data.shape[2]
level_slider = widgets.IntSlider(min=-1024, max=1024, step=1, value=40, description='Level')
width_slider = widgets.IntSlider(min=1, max=2000, step=1, value=400, description='Width')
slice_slider = widgets.IntSlider(min=0, max=num_slices - 1, step=1, description='Slice Index')
# Create a scrollable widget
slice_slider = widgets.IntSlider(
    value=slices_with_1[0],
    min=min(slices_with_1),
    max=max(slices_with_1),
    step=1,
    description='Slice:',
    continuous_update=False
)

# Create an interactive widget
widgets.interactive(plot_image_and_mask, slice_index=slice_slider, level=level_slider, width=width_slider)

Using device: cuda


interactive(children=(IntSlider(value=26, continuous_update=False, description='Slice:', max=34, min=26), IntS…